<br/>
<h1 align="center"> <span style = 'color:red'> Detecção de Spam com Multinomial NB
</span> </h1>
<br/>




* O naive bayes é muto utilizado para classificação de bases textuais



* Base textuais devem ser representandas em um formato que o algoritmo possa lidar



* A metodologia utilizada é processar o texto e após utilizar algum representação numérica ou modelo de linguagem



*  Existem uma série de modelos de linguagem, nos proximos notebooks vamos tratar desse modelos




<br/>
<h3 align="left"> <span style = 'color:green'> Utilizando a biblioteca NLTK para processamento de texto    </span> </h3>
<br/>


* A biblioteca NLTK faz toda a parte do processamento do texto, é muito bem adapatada ao idioma ingles, pouco adaptada ao idioma portugues.








<h2 align="left"> <span style = 'color:green'>Características da base </span> </h2>
<br/>

<h3 align="left"> <span style = 'color:black'>  1. Contexto   </span> </h3>

* The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. 

* It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam.
<br/>

<h3 align="left"> <span style = 'color:black'>  2 .Conteúdo   </span> </h3>

* The files contain one message per line. 


* Each line is composed by two columns: v1 contains the label (ham or spam) and v2 contains the raw text.


* This corpus has been collected from free or free for research sources at the Internet:


* *A collection of 425 SMS spam messages was manually extracted from the Grumbletext Web site. This is a UK forum in which cell phone users make public claims about SMS spam messages, most of them without reporting the very spam message received. The identification of the text of spam messages in the claims is a very hard and time-consuming task, and it involved carefully scanning hundreds of web pages*






<br/>     

<br/>     

<h3 align="left"> <span style = 'color:green'>  link para a competição no Kaggle </span> </h3>



* Esse é o link para acessar a base de dados e o notebook, mas é necessário que se faça o cadastro - "CountVectorizer and Tfidf strategies" [Kaggle Competitions]( https://www.kaggle.com/pceccon/countvectorizer-and-tfidf-strategies/data).

<br/>     



 

<br/>
<br/>
<h3 align="left"> <span style = 'color:green'> Importando bibliotecas    </span> </h3>


In [1]:
import numpy as np 
import pandas as pd 

<br/>
<h3 align="left"> <span style = 'color:green'>  Carregando arquivo   </span> </h3>


<br/>

* Detalhe: o idioma deste texto é o ingles logo o char-set é **latin1**, caso fosse o portugues seria **utf-8**





In [3]:
data = pd.read_csv("spam.csv" ,encoding='latin1')

<br/>
<h3 align="left"> <span style = 'color:green'>  Mostrando os 5 primeiros registros   </span> </h3>


In [4]:
# v1 é o label - rotulo a ser classificado
# v2 é o email - texto a ser classificado em spam e ham (not-spam)
# as outras variáveis estão com pampo vazio - NaN e devem ser retiradas


# mostrando os 5 primeiros registros
data.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


<br/>
<h3 align="left"> <span style = 'color:green'>   Mostrando informações  sobre a base de dados </span> </h3>


In [3]:
# informaçãoes sobre a base
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.7+ KB


<br/>
<h3 align="left"> <span style = 'color:green'>  Nomeando a entrada e saída de dados   </span> </h3>


In [6]:
# atribuindo valores a x - email e a y - label (classe)

X = data["v2"].values
y = data["v1"].values

<br/>
<h3 align="left"> <span style = 'color:green'> Importando a biblioteca NLTK    </span> </h3>



*  Atenção:


    * Caso a bliblioeca não esteja instalada, retirar o simbolo #  rodar e instalar

In [6]:
#import nltk
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

<br/>
<h3 align="left"> <span style = 'color:green'>  Processamento do texto   </span> </h3>


* Importando a biblioteca que faz a remoção de stopwords


    * Stopwords são palavras que possuem alta frequencia nos documentos e em toda a coleção de documentos
    
    * Normalmente essas palavras não auxiliam na classifcação


<br/>


* Importando a biblioteca regex


    * responsável por realizar operações sobre strings no python




In [7]:
# para retirar stopwords
# stopword são palavras que apresentam alta frequencia em documentos e na coleção (corpus = coleção de documentos)

from nltk.corpus import stopwords 

# bibliotece regex para manipulação de string
import re

<h3 align="left"> <span style = 'color:green'> Pequeno processamento do texto    </span> </h3>
<br/>



* Remoção de stopword de uma lista prévia


* Lematizador (stemmer) separa o radical da palavra ex: trabalhador ficaria trab


* Converte todas as palavras para minúscula

In [10]:
from nltk.stem import PorterStemmer 

# biblioteca para lematização, retira o sufixo das palavras
ps = PorterStemmer()



# removendo as stopwords
stopword = set(stopwords.words('english'))



# substituindo as palavra em maiúscula para muniúscula
X = [re.sub('[^a-zA-Z]',' ',doc) for doc in X ]
document = [doc.split() for doc in X ]




#recriando o arquivo apenas com os stemm e sem stopword, todas as palavras em minúscula
def convert(doc):
    temp = []
    for word in doc:
        if word.lower() not in stopword:
            temp.insert(len(temp),ps.stem(word).lower())
    return temp        

# convertendos os conjuntos de documentos (emails)
document = [convert(doc) for doc in document] 


# convertendo em uma variável documento separada por espaços, pois a variável X que recebeu os emails tem um formato 
# que não é apropriado para ser lido pelo método countvectorizer
document = [" ".join(doc) for doc in document]

<h3 align="left"> <span style = 'color:green'>  Método  CountVectorizer com o classificador  MultinomialNB </span> </h3>
<br/>



* O método **Countvectorizer** é uma das formas de representar numericamente uma base textual


    * Método baseado na frequencia que uma palavra (token = termo) tem em um documento





* **No notebook 6  - Utilizando Countvectorize e TD-IDF para classificação de texto** tem explicação desses métodos

In [11]:
from sklearn.model_selection import train_test_split


# transformando o texto em uma representação vetorial 
from sklearn.feature_extraction.text import CountVectorizer


# Importando o classificador
from sklearn.naive_bayes import MultinomialNB


<br/>
<h3 align="left"> <span style = 'color:green'>Dividindo a base     </span> </h3>

In [12]:
X_train , X_test , y_train , y_test = train_test_split(document,y)

<br/>
<h3 align="left"> <span style = 'color:green'>  Criando modelo de linguagem e treinando   </span> </h3>

In [13]:
# transformndo o documento em vetores com no máximo 1000 características, ou seja palavras diferentes
cv = CountVectorizer(max_features = 1000)



# transformando os vetores de palavras em uma matriz de treinamento
a = cv.fit_transform(X_train) 


# criando o vocabulario , indice de palavras
vocab = cv.get_feature_names()


# transformando os vetores de palavras em uma matriz de teste
b= cv.transform(X_test) 


<br/>
<h3 align="left"> <span style = 'color:green'>  Criando modelo de classificador e treinando   </span> </h3>

In [14]:
# criando o modelo de classificação

clf = MultinomialNB()

# treinamento do modelo
clf.fit(a,y_train)

# mostrando o valor da acurácia
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

print(40 * '*')
print('Acurácia é de =',round(clf.score(b,y_test)*100,2),'%')
print(40 * '*')

****************************************
Acurácia é de = 97.92 %
****************************************
